In [1]:
import sys
import plotly.graph_objects as go
sys.path.append("..") 
from orm.mongo_client import FoodMongoClient
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

In [2]:
tags_k2v = {}
ingredients_k2v = {}
dataset = []
with FoodMongoClient() as mngcl:
    tags = mngcl.find_all('tags')
    ingredients = mngcl.find_all('ingredients')
    recepes = mngcl.find_all('recipes')
    recepes_k2v = dict([(rec['id'], rec['name']) for rec in recepes])
    recepes_v2k = dict([(rec['name'], rec['id']) for rec in recepes])
    for rec in recepes:
        dataset.append(['t'+str(tag) for tag in rec['tags']] + 
                       ['r'+str(rec['id'])] + 
                       ['i'+str(ing['id']) for ing in rec['ingredients']])

In [3]:
recepes_v2k

{'Тонкое тесто для\xa0пиццы': '22835',
 'Брауни с\xa0темным шоколадом': '28338',
 'Классический греческий салат (Horiatiki)': '21993',
 'Шоколадные маффины с\xa0кусочками шоколада': '36024',
 'Торт «Сметанник»': '23485',
 'Тирамису с\xa0сыром маскарпоне': '27604',
 'Тушеная говядина с\xa0морковью и\xa0луком': '40151',
 'Пшенная каша на\xa0молоке': '39801',
 'Мясо по‑французски с\xa0майонезом': '32718',
 'Торт «Муравейник» со\xa0сгущенным молоком': '37971',
 'Лагман': '37230',
 'Жульен с\xa0курицей и\xa0грибами': '45809',
 'Капрезе с\xa0бальзамическим уксусом': '29115',
 'Куриное филе с\xa0грибами и\xa0сливками': '34995',
 'Стейк из\xa0тунца с\xa0гарниром из\xa0брюссельской…': '30378',
 'Рассольник с\xa0перловкой': '41960',
 'Паста с\xa0креветками в\xa0сливочно-чесночном соусе': '55603',
 'Простой бисквит': '15039',
 'Тесто для\xa0пельменей': '37069',
 'Рулька свиная, запеченая в\xa0духовке': '19747',
 'Лапша удон с\xa0курицей и\xa0овощами': '49545',
 'Салат из\xa0запеченной свеклы с\xa

In [108]:
model = Word2Vec(dataset, size=32,
        window=5,
        min_count=1,
        workers=4,
        iter=10)
#model = Word2Vec.load("../models/word2vec128.model")

In [109]:
epoch_by_step, step_numbers = 10, 10
for i in range(step_numbers):
    recipe_keys = [key for key in model.wv.vocab if key[0] == 'r']
    recipe_vectors = np.array([model.wv[key] for key in recipe_keys])
    recipe_pca_vectors = PCA(n_components=2).fit_transform(recipe_vectors)
    fig = go.Figure(data=go.Scatter(x=recipe_pca_vectors[:, 0],
                            y=recipe_pca_vectors[:, 1],
                            mode='markers',
                            text=[recepes_k2v[key[1:]] for key in recipe_keys])) # hover text goes here
    fig.update_layout(title=f'Recipes: {i*epoch_by_step} epoche')
    fig.show()
    model.train(dataset, total_examples=len(dataset), epochs=epoch_by_step)


In [6]:
model.save("../models/word2vec32_sg.model")

In [110]:
sim = dict(model.wv.most_similar(positive=[
#     f"r{recepes_v2k['Тыквенный торт']}", 
#     f"r{recepes_v2k['Торт «Ленинградский»']}", 
#     f"r{recepes_v2k['Яблочная быстрая шарлотка']}", 
    f"r{recepes_v2k['Жареная хамса']}",
],  topn = 100))
for key in sim:
    if 'r' == key[0]: print(recepes_k2v[key[1:]], sim[key])

Жареная ставрида 0.7838881611824036
Шпроты с лимоном 0.7824252843856812
Аджапсандали 0.7815493941307068
Бабагануш с кунжутом 0.765198290348053
Вареная кукуруза 0.7436949014663696
Аджарские хачапури 0.7388354539871216
Салат с жареным баклажаном и перцем 0.7263115644454956
Баклажаны жареные с гранатом и зеленью 0.7260326147079468
Имеретинские хачапури 0.7199420928955078
Мегрельские хачапури 0.7119855880737305
Баклажаны с соусом бажи 0.7117390632629395
Манник 0.7049475312232971
Айоли 0.7032620906829834
Квашеная капуста с тмином 0.700642466545105
Осетрина в гранатовом соусе 0.7001404762268066
Завтрак для ленивых 0.699601411819458
Хала 0.6932927370071411
Воздушное тесто для пирожков 0.6930176615715027
Табуле 0.692970335483551
Соус сацебели 0.6907715201377869
Беляши 0.690460205078125
Соленый муксун 0.6861404180526733
Жареный сулугуни 0.6808995008468628
Хумус со шпинатом 0.6797597408294678
Блинчики на воде 0.6735529899597168
Мягкие плюшки с сахаром 0.6641979217529297
Салат из перца по‑баскски

In [111]:
recipe_keys = [key for key in model.wv.vocab if key[0] == 'r']
recipe_vectors = np.array([model.wv[key] for key in recipe_keys])

tree_id2rec_key = dict([(i, recipe[1:]) for i, recipe in enumerate(recipe_keys)])
rec_key2tree_id = dict([(recipe[1:], i) for i, recipe in enumerate(recipe_keys)])

rec_name2tree_id = dict([(recepes_k2v[recipe[1:]], i) for i, recipe in enumerate(recipe_keys)])
tree_id2rec_name = dict([(i, recepes_k2v[recipe[1:]]) for i, recipe in enumerate(recipe_keys)])

In [116]:
recipe_names = ['Плов «Чайханский»', 
                'Торт «Ленинградский»', 
                'Яблочная быстрая шарлотка', 
                'Торт «Полет»', 
                'Торт «Муравейник» со\xa0сгущенным молоком',
                'Ореховая аджика',
                'Жареная камбала',
                'Макароны по‑флотски',
                'Манник',
                'Соус сацебели',
                'Кучмачи',
                'Чахохбили',
               ]
vectors = list(map(lambda x: recipe_vectors[rec_name2tree_id[x]], recipe_names))

from sklearn.cluster import DBSCAN
import pprint
clustering = DBSCAN(eps=1.3, min_samples=2).fit(vectors)
clusters_set = set(clustering.labels_)

#pprint.pprint(sorted(list(zip(recipe_names, clustering.labels_)), key=lambda x: x[1]))

data_for_recomendations = dict(zip(clusters_set, [
    [rec for i, rec in enumerate(recipe_names) if clustering.labels_[i] == label] for label in clusters_set]))

pprint.pprint(data_for_recomendations)

{-1: ['Торт «Ленинградский»', 'Ореховая аджика', 'Макароны по‑флотски'],
 0: ['Плов «Чайханский»', 'Соус сацебели', 'Кучмачи', 'Чахохбили'],
 1: ['Яблочная быстрая шарлотка',
     'Торт «Полет»',
     'Торт «Муравейник» со\xa0сгущенным молоком',
     'Жареная камбала',
     'Манник']}


In [117]:
from sklearn.neighbors import KDTree
from operator import itemgetter
tree = KDTree(recipe_vectors, leaf_size=2)

recomendations = {}

for label, names in data_for_recomendations.items():
    if label == -1: continue
    
    
    dist, ind = tree.query(
        [
            np.mean(
                list(map(lambda x: recipe_vectors[rec_name2tree_id[x]], names))
                , axis=0)
        ], k=20)
    
    keys = itemgetter(*ind[0])(tree_id2rec_key)
    values = itemgetter(*ind[0])(tree_id2rec_name)
    
    recomendations[label] = list(zip(keys, values))
    

pprint.pprint(recomendations)

{0: [('47838', 'Кучмачи'),
     ('17430', 'Индейка под\xa0соусом из\xa0грецких орехов'),
     ('126166', 'Чахохбили'),
     ('126527', 'Сациви'),
     ('31565', 'Курица с\xa0ежевикой'),
     ('17109', 'Соус сацебели'),
     ('14593', 'Лобио'),
     ('31185', 'Плов «Чайханский»'),
     ('25199', 'Кюфта-бозбаш'),
     ('21751', 'Азу\xa0по‑татарски'),
     ('31243', 'Аджарские хачапури'),
     ('19319', 'Осетрина в\xa0гранатовом соусе'),
     ('126178', 'Ленинградский рассольник'),
     ('125803', 'Фасолевый суп с\xa0копченостями'),
     ('93970', 'Энчилада с\xa0зерновым творогом и\xa0курицей'),
     ('50983', 'Жареные бычки'),
     ('125381', 'Аджапсандали'),
     ('93284', 'Пико-де-гальо'),
     ('31240', 'Имеретинские хачапури'),
     ('125353', 'Пастилла')],
 1: [('15214', 'Блинчики с\xa0творожной начинкой'),
     ('16149', 'Творожный кекс с\xa0яйцами'),
     ('30707', 'Шоколадно-апельсиновое печенье'),
     ('15039', 'Простой бисквит'),
     ('16014', 'Тонкие блины на\xa0молоке'),
  

In [118]:
import pickle as pkl

with open('../models/w2v_kdtree.pkl', 'wb') as kdtreef:
    pkl.dump(tree, kdtreef)
with open('../models/treeid2reckey.dict', 'wb') as dictf:
    pkl.dump(tree_id2rec_key, dictf)
with open('../models/reckey2treeid.dict', 'wb') as dictf:
    pkl.dump(rec_key2tree_id, dictf)
with open('../models/treeid2recname.dict', 'wb') as dictf:
    pkl.dump(tree_id2rec_name, dictf)
with open('../models/recname2treeid.dict', 'wb') as dictf:
    pkl.dump(rec_name2tree_id, dictf)
with open('../models/recipe_vectors.pkl', 'wb') as dictf:
    pkl.dump(recipe_vectors, dictf)